In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Fact Extraction and VERification task

In [2]:
import os
import numpy as np
import csv
import pandas as pd
import json
import string
import collections
import itertools
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
!pip install jsonlines
import jsonlines as jl
#!pip install wikipedia
#import wikipedia

In [10]:
import torch
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
#If the file is downloaded to the format .json1 you have to rename the file and remove the "1" at the end.
#Otherwise it's impossible to read the file
datafile_fever = '/content/drive/My Drive/Colab Notebooks/data/train.json'
datafile_shared_task = '/content/drive/My Drive/Colab Notebooks/data/shared_task_dev.json'
wiki_pages_folder = '/content/drive/My Drive/Colab Notebooks/data/wiki-pages/wiki-pages'

In [6]:
df_train = pd.read_json(datafile_fever, lines=True)
df_train.shape

(145449, 5)

In [7]:
df_task = pd.read_json(datafile_shared_task, lines=True)
df_task.shape

(19998, 5)

Importation of wiki pages data

In [0]:
from os.path import isfile, join
files = [f for f in os.listdir(wiki_pages_folder) if isfile(join(wiki_pages_folder, f))]

In [9]:
with jl.open(wiki_pages_folder+"/"+files[10]) as f:
  for line in tqdm(f.iter()):
      print(line["id"])

952it [00:00,  3.13it/s]

Anna_Szabolcsi
Arilda_of_Oldbury
Arai_En_305-il_Kadavul
Antonio_Pimentel_de_Prado
André_Hornez
Army_Men-COLON-_Sarge's_Heroes
Antoine_Groignard
Anni_Holdmann
Apostolos_Kathiniotis
Anglo-American_music
Armalite_AR-30
Armagh_railway_station
Andrei_Kalaychev
Anasuyabai_Kale
Angel_Coulby
Antonie_Misset
Arab_Hasan
Anacrusis_epidicta
Arrai_TV
Ancient_Greek_novel
Antonians_Sports_Club
Anaplastic_large-cell_lymphoma
Appellation_d'origine_protégée_-LRB-Switzerland-RRB-
Anjali_Sudhakar
Andrew_Mackay_-LRB-British_Army_officer-RRB-
Antoni
Andrucha_Waddington
Anchu_Sundarikal
Anawarkhi
Andrew_John_Berger
Arnspitze_Group
Archer_-LRB-season_5-RRB-
Anemia_-LRB-beetle-RRB-
Antoine_de_Saint_Exupéry_Airport
Arthur_Gordon_Matthew
Animaland_Park
Ansty
Anastrepha
Arrowsmith,_Illinois
Anna_Minton
Armin_Scholz
Angangueo
Aphrodite_-LRB-Hercules-COLON-_The_Legendary_Journeys_and_Xena-COLON-_Warrior_Princess-RRB-
Ardit_Zenuni
Annette_Melton
Ann_and_Robert_H._Lurie_Foundation
Anandalok
Andy_McGill
Arif_Babayev
Ar

1792it [00:01,  6.39it/s]

Arslan_Tash_amulets
Andrei_Kotsur
Arseny_Sokolov
Arthur_Sasse
Ann_Althouse
Anne_Poulet
Ardulfurataini_-LRB-national_anthem-RRB-
Andilla
Archbishop_Denis_O'Connor_Catholic_High_School
Anna_DiMera
Anglican_Church_of_Mexico
Andrew_B._Cooke_House
Arthur's_Day
April_1975
Armen_Babalaryan
Andrew_Thomson_-LRB-Australian_footballer-RRB-
Anselm_of_Laon
Archbishop_Rummel_High_School
Artec_3D
Arrowina
Antony_James
Antônio_do_Carmo_Cheuiche
Arctic_Velvet
Arne_Brun_Lie
Anaís
Arthur_Aston_-LRB-army_officer-RRB-
Apartments_and_Flats_of_Downtown_Indianapolis_Thematic_Resources
Ariola_ransonneti
Arkansas_River
Aqigssiaq_Maniitsoq
Appanapalle
Angels_of_the_Street
Arne-Johan_Henrichsen
Arjantin_Square
Archibald_Cary
Annibale_Bentivoglio
Anklam-Land
Anthony_Alix
Andy_Frampton
Anacrusis_stapiana
Anastasius_Germonius
Armida_-LRB-Weir-RRB-
Army_Reserve_Sustainment_Command
Animal_Miracles
Artesia_High_School_-LRB-California-RRB-
Andranomanelatra
Anacron
Andrew_Jackson
Ariel_Lin
Andrew_Heafitz
Anoplognathus_sm

2690it [00:01, 13.01it/s]

...

Aspinoceras
Bahçeşehir_Koleji
Atriplex_leucophylla
Arturo_Armando_Molina
Arundel_and_Shoreham_by-election,_1954
Austria_in_the_Eurovision_Song_Contest_1999
Australian_Fencing_Federation
Avant's_Cities_Service_Station
Auspitz
Baltimore_Mills_Historic_Archaeological_Site
Babala
Artur_Grottger
Asius_-LRB-mythology-RRB-
Ash,_Kip_and_Luttsy_with_Susie_O'Neill
Barbara_Sass
Banjo_Eyes
Bardigiano
Barend_Graat
Barbara_Terhal
Bank_of_Tamil_Eelam
B*Witched_discography
Asher_yatzar
Averch–Johnson_effect
Autreppes
BUAA_Posion_Bee
Baboucarr-Blaise_Jagne
Atriplex_valdesii
Bamboo_Mañalac
Barbara_Mitchelhill
Avra_M._Warren
Australian_rules_football_in_the_Middle_East
Balcombe
B._C._Patil
Aurora_Perrineau
Attached_Senior_School_of_Shandong_Normal_University
Babiniec,_Lesser_Poland_Voivodeship
Auxiliary_ship
Athletics_at_the_1975_South_Pacific_Games
BMU_Lebanese_French_University
Asoka_Weerasinghe
Bahía_Posesión
Augustin_Nsanze
Auberives-en-Royans
Asaphocrita_blattae
Backseat_Film_Festival
Associazione_S

48793it [00:11, 4359.84it/s]


Arístides_Mejía
Atomic_Energy_Research_Establishment_School_and_College
Avchala
Bandini-Maserati_1500
Asgarov
Barbodes_tras
Assistant_Secretary_of_the_Navy
Attached_gas-check
Bare,_Kraljevo
Aura_Hydroelectric_Power_Station
Bar_Rud,_Razavi_Khorasan
Ballybunion_Golf_Club
Bakel_Department
Barclay_Fowell_Buxton
Bachelor_of_Environmental_Science
Autoroute_73
Australian_Cup
Bald_Hill_Township,_Jefferson_County,_Illinois
B.M.S._College_of_Engineering
Austin_Rover_World_Rally_Team
Arturo_Barba
Arzillières-Neuville
Avia_BH-16
Bank_F.C.
Azerbaijan_at_the_2013_World_Aquatics_Championships
Ayas_Jan
Bandra_Kurla_Complex
Australian_women's_cricket_team_in_England_in_2005
Avksily
Badger_Pot
Bach_Media
Ater_Majok
Azimzhan_Askarov
Artline
Badminton_at_the_1994_Asian_Games_–_Men's_doubles
BL_6_inch_naval_guns_Mk_XIII_–_XVIII
Athens,_New_York_-LRB-disambiguation-RRB-
Ayyavazhi
Ashirbek_Torebayuly_Sygai
Arturo_Peniche
Baglan_-LRB-electoral_ward-RRB-
Athletics_at_the_2011_Summer_Universiade_–_Women's_100_

50000it [00:11, 4302.45it/s]

Arujá
Back_to_Bacharach
Ayi_language
Baby_Chaos
Baranowiec
Bahia_bigelovii
Bahlool
Banksia_subser._Leptophyllae
Ayee_Milan_Ki_Raat
Aureliano_Fernández-Guerra
Balin,_Inowrocław_County
Athletics_at_the_2011_Pacific_Games
Available_Light_-LRB-album-RRB-
Bagsecg
Athletics_at_the_2010_South_American_Games_–_Women's_1500_metres
Baldwin_Lake
Asociación_del_Sello_de_Productos_de_Comercio_Justo
Augustin_Lesage
Australian_Tour_EP
Asperdaphne_sepulta
Average_worker's_wage
Bar_Keepers_Friend
Bancroft
As-Salam_College_-LRB-English_Mission_College-RRB-,_Cairo
Asplenium_milnei
BYU_Geospatial_Lab
Badminton_Club
Ayushita
Azerbaijan_Women's_Handball_Championship
BDO_International
Ashfaq_Ahmed_-LRB-cricketer,_born_1985-RRB-
Bagzan
Aubrey_Beauclerk_-LRB-cricketer-RRB-
Artur_Zapadnya
Atyrau_Region
Awele_Maduemezia
Baalah
Arvand_Free_Zone
Bakka'in
Austral_snipe
Ashfield_Boys_High_School
Banner_of_Truth_Trust
Atgāzene_Station
Bahia_Mouhtassine
Aryl-alcohol_oxidase
BMW_M102
Balashov_-LRB-town-RRB-
Bardzice
Av

## Data exploration

In [11]:
df_train['evidence'][5]

[[[151831, 166598, 'Homeland_-LRB-TV_series-RRB-', 0],
  [151831, 166598, 'Prisoners_of_War_-LRB-TV_series-RRB-', 0]]]

In [12]:
#df_task.head()
print(df_train['claim'][0])
print(df_train['evidence'][0][0][0][3])

Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
7


In [13]:
{"id": "Nikolaj_Coster-Waldau", "text": "Nikolaj Coster-Waldau -LRB- -LSB- ne\u0261\u030aola\u026a\u032f k\u02b0\u028csd\u0325\u0250 \u02c8\u028bald\u0325\u0251\u028a\u032f -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter . He graduated from Danish National School of Theatre in Copenhagen in 1993 . Coster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- . Since then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- .   In the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon . He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot . He became widely known to a broad audience for his current role as Ser Jaime Lannister , in the HBO series Game of Thrones . In 2017 , he became one of the highest paid actors on television and earned # 2 million per episode of Game of Thrones . ", "lines": "0\tNikolaj Coster-Waldau -LRB- -LSB- ne\u0261\u030aola\u026a\u032f k\u02b0\u028csd\u0325\u0250 \u02c8\u028bald\u0325\u0251\u028a\u032f -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter .\n1\tHe graduated from Danish National School of Theatre in Copenhagen in 1993 .\tDanish National School of Theatre\tDanish National School of Theatre and Contemporary Dance\tCopenhagen\tCopenhagen\n2\tCoster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- .\tNightwatch\tNightwatch (1994 film)\n3\tSince then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- .\tHeadhunters\tHeadhunters (film)\tA Thousand Times Good Night\tA Thousand Times Good Night\n4\t\n5\t\n6\tIn the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon .\tBlack Hawk Down\tBlack Hawk Down (film)\tGary Gordon\tGary Gordon\n7\tHe then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot .\tFox television\tFox Broadcasting Company\tNew Amsterdam\tNew Amsterdam (TV series)\tVirtuality\tVirtuality (TV series)\n8\tHe became widely known to a broad audience for his current role as Ser Jaime Lannister , in the HBO series Game of Thrones .\tJaime Lannister\tJaime Lannister\tHBO\tHBO\tGame of Thrones\tGame of Thrones\n9\tIn 2017 , he became one of the highest paid actors on television and earned # 2 million per episode of Game of Thrones .\tGame of Thrones\tGame of Thrones\thighest paid actors on television\tList of highest paid American television stars\n10\t"}

{'id': 'Nikolaj_Coster-Waldau',
 'lines': "0\tNikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter .\n1\tHe graduated from Danish National School of Theatre in Copenhagen in 1993 .\tDanish National School of Theatre\tDanish National School of Theatre and Contemporary Dance\tCopenhagen\tCopenhagen\n2\tCoster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- .\tNightwatch\tNightwatch (1994 film)\n3\tSince then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- .\tHeadhunters\tHeadhunters (film)\tA Thousand Times Good Night\tA Thousand Times Good Night\n4\t\n5\t\n6\tIn the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon .\tBlack Hawk Down\tBlack Hawk

In [14]:
df_train[5:10].head()

,id,verifiable,label,claim,evidence
5,129629,VERIFIABLE,SUPPORTS,Homeland is an American television spy thrille...,"[[[151831, 166598, Homeland_-LRB-TV_series-RRB..."
6,149579,NOT VERIFIABLE,NOT ENOUGH INFO,Beautiful reached number two on the Billboard ...,"[[[173384, None, None, None]]]"
7,229289,NOT VERIFIABLE,NOT ENOUGH INFO,Neal Schon was named in 1954.,"[[[273626, None, None, None]]]"
8,33078,VERIFIABLE,SUPPORTS,The Boston Celtics play their home games at TD...,"[[[49158, 58489, Boston_Celtics, 3]], [[49159,..."
9,6744,VERIFIABLE,SUPPORTS,The Ten Commandments is an epic film.,"[[[23513, 28977, The_Ten_Commandments_-LRB-195..."


In [15]:
df_train['evidence'] = df_train['evidence'].replace("-LRB-","(")
df_train['evidence'] = df_train['evidence'].replace("-RRB-",")")
df_train[0:10].head()

,id,verifiable,label,claim,evidence
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271..."
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]"
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]"
4,83235,NOT VERIFIABLE,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,"[[[100277, None, None, None]]]"


In [0]:
#wikipedia.suggest('Homeland_-LRB-TV_series-RRB-')

# Find evidence

In [0]:
#path, dirs, files = next(os.walk(wiki_pages_folder))
file_count = len(files)

def findEvidence(evidence,verifiable):
  found = False
  for i in tqdm(range(len(files))):
    with jl.open(wiki_pages_folder+"/"+files[i]) as f:
      for j, article in enumerate(f.iter()):   
        # wiki[283015][title_id] = wiki['id'][title_id].split("\n")
        # wiki[283015][title_id] = wiki['id'][title_id].replace(" ", "_")
        if article['id']==evidence:
          print(article['id'])
          print("file : ", files[i]," and article number : ", j)
          found = True
          break
    if found:
      break
    

In [18]:
data = train_findEvidence(df_train['evidence'][0:5],df_train['verifiable'][0:5])

NameError: ignored

In [19]:
df_train['evidence'][0:5][0]

[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
  [92206, 104971, 'Fox_Broadcasting_Company', 0]]]

In [20]:
wiki = []
with jl.open(wiki_pages_folder+'/wiki-001.jsonl') as f:
  for line in f.iter():
      wiki.append(line)
print("wiki shape : (", len(wiki), ",", len(wiki[0]), ")")

wiki shape : ( 50000 , 3 )


In [21]:
# wiki['id'][283015] = wiki['id'][283015].split("\n")
# wiki['id'][283015] = wiki['id'][283015].replace(" ", "_")
print(wiki[28015]["id"])

1992_Pacific_typhoon_season


In [22]:
findEvidence('1992_Pacific_typhoon_season',0)

  0%|          | 0/109 [00:00<?, ?it/s]

1992_Pacific_typhoon_season
file :  wiki-001.jsonl  and article number :  28015


In [0]:
for i in tqdm(range(len(df_train['evidence']))):
  for j in range(len(df_train['evidence'][i])):
    for k in range(len(df_train['evidence'][i][j])):
      findEvidence(df_train['evidence'][i][j][k][2],df_train['evidence'][i][j][k][3])

 58%|█████▊    | 63/109 [01:54<01:25,  1.86s/it]

In [0]:
b = wikipedia.suggest("association_football in 1928")
print(b)

None


In [0]:
a = wikipedia.search(' Nikolaj_Coster-Waldau')
print(a)

['Nikolaj Coster-Waldau', 'Nukâka Coster-Waldau', 'Shot Caller (film)', 'Small Crimes', 'Gods of Egypt (film)', 'The Silencing', 'Domino (2019 film)', 'The Other Woman (2014 film)', 'A Second Chance (2014 film)', 'Nightwatch (1994 film)']


In [0]:
b = a[0].replace(" ", "_")
#print(b)
article = (wikipedia.page('Nikolaj_Coster_Waldau').content).split(".")
print(article[6])

 He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam (2008), as well as appearing as Frank Pike in the 2009 Fox television film Virtuality, originally intended as a pilot


In [0]:
wikipedia.suggest('Nikolaj_Coster-Waldau')

'nikolaj coster waldau'

In [0]:
#import re
def train_findEvidence(evidence,verifiable):
#trouver la ou les phrases
#si plusieurs, les concatener
#ensuite mettre l'evidence dans un data_frame en gardant les id 
#si le titre contient des parentheses on a l'a dans le cul faut supprimer

  dataset = []
  for i in tqdm(range(len(evidence))):
    if verifiable[i]=="VERIFIABLE":
      for j in range(len(evidence[i])):
        sentence = ""
        for k in range(len(evidence[i][j])):
          url = evidence[i][j][k][2]
          if "-LRB-" not in url:
            line = evidence[i][j][k][3]
            #print("url : ",type(url))
            url = url.replace("-","_")
            print(url)
            #suggest = wikipedia.suggest(url)
            #if suggest !=None:
            try:
              article = (wikipedia.page(url).content).split(".")
              break
            except PageError:
              sentence += ' '+article[line]
          dataset.append([evidence[i][j][k][0],evidence[i][j][k][1],sentence])
  return dataset

In [0]:
data = train_findEvidence(df_train['evidence'][0:110],df_train['verifiable'][0:110])
print(data)






  0%|          | 0/110 [00:00<?, ?it/s]

Nikolaj_Coster_Waldau







  1%|          | 1/110 [00:00<01:23,  1.30it/s]

Roman_Atwood
Roman_Atwood







  2%|▏         | 2/110 [00:02<01:46,  1.01it/s]

...






 89%|████████▉ | 98/110 [01:30<00:08,  1.41it/s]

Khmer_Empire







 92%|█████████▏| 101/110 [01:31<00:05,  1.74it/s]

Peter_Davison


NameError: ignored

In [0]:
df_train['evidence'][100:110]

100                  [[[66965, 77763, Khmer_Empire, 0]]]
101    [[[199623, 209302, On_the_Road_-LRB-film-RRB-,...
102                       [[[133857, None, None, None]]]
103    [[[106250, 119689, Paramore_-LRB-album-RRB-, 2]]]
104    [[[171118, 184635, Peter_Davison, 1]], [[17111...
105    [[[136980, 151810, 21_Jump_Street_-LRB-film-RR...
106    [[[247949, 248240, Philadelphia_Museum_of_Art,...
107    [[[158017, 172644, Dustin_Hoffman, 6], [158017...
108     [[[260555, 258945, Deadpool_-LRB-film-RRB-, 1]]]
109                       [[[247476, None, None, None]]]
Name: evidence, dtype: object

In [0]:
print(data[5])

[180804, 193183, ' \n\n\n== Discography ==\n\n\n=== Studio albums ===\n\n\n=== Guest appearances ===\n\n\n=== Unreleased/shelved ===\nUnapologetic (2010)\n\n\n== Filmography ==\n\n\n=== Film roles ===\n\n\n=== Television roles ===\n\n\n=== Web ===\n\n\n=== Music video appearances ===\n\n\n== Awards and nominations ==\nNote: The year given is the year of the ceremony\n\n\n== References ==\n\n\n== External links ==\n\n{}}\nAdrienne Bailon on IMDb']


In [0]:
# wikipedia.search("Nikolaj_Coster_Waldau")
a = (wikipedia.page("Nikolaj_Coster_Waldau").content).split(".")
print(a[0])

Nikolaj Coster-Waldau (Danish pronunciation: [ˈne̝koˌlɑjˀ ˈkʰʌstɐ ˈvæltɑw]; born 27 July 1970) is a Danish actor, producer and screenwriter


In [0]:
(wikipedia.page('Peter_Davison').content).split(".")

PageError: ignored

In [0]:
wikipedia.search('Peter_Davison')

['Peter Davison',
 'Peter Davison (disambiguation)',
 'Fifth Doctor',
 'Peter Davison (professor)',
 'Peter Davison (footballer)',
 'Peter Davison (poet)',
 'All Creatures Great and Small (TV series)',
 'The Five(ish) Doctors Reboot',
 'Elizabeth Morton',
 'David Tennant']

In [0]:
!pip3 install wikipedia-api

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-cp36-none-any.whl size=13462 sha256=2d25eb83129e81f8d46da042e6c3c47640179518d86f47561c481f0edb512e45
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built wikipedia-api


In [0]:
import wikipediaapi as wikipedia

In [0]:
wiki = wikipedia.Wikipedia('en')
# page_py = wiki_wiki.page('On_the_Road_(film)')
# print("Page - Title: %s" % page_py.title)
#     # Page - Title: Python (programming language)

# print("Page - Summary: %s" % page_py.summary)
    # Page - Summary: Python is a widely used high-level programming language for

In [0]:
#import re
def train_findEvidence(evidence,verifiable):
#trouver la ou les phrases
#si plusieurs, les concatener
#ensuite mettre l'evidence dans un data_frame en gardant les id 
#si le titre contient des parentheses on a l'a dans le cul faut supprimer

  dataset = []
  for i in tqdm(range(len(evidence))):
    if verifiable[i]=="VERIFIABLE":
      for j in range(len(evidence[i])):
        sentence = ""
        for k in range(len(evidence[i][j])):
          url = evidence[i][j][k][2]
          #if "-LRB-" not in url:
          line = evidence[i][j][k][3]
          #print("url : ",type(url))
          url = url.replace("-LRB-","(")
          url = url.replace("-RRB-",")")
          url = url.replace("-","_")
          print(url)
          #suggest = wikipedia.suggest(url)
          #if suggest !=None:
          article = wiki.page(url).summary.split(".")
          sentence += ' '+article[line]
        dataset.append([evidence[i][j][k][0],evidence[i][j][k][1],sentence])
  return dataset

In [0]:
data = train_findEvidence(df_train['evidence'][0:110],df_train['verifiable'][0:110])
print(data)








  0%|          | 0/110 [00:00<?, ?it/s]

Nikolaj_Coster_Waldau









  1%|          | 1/110 [00:00<00:47,  2.29it/s]

Fox_Broadcasting_Company
Roman_Atwood









  2%|▏         | 2/110 [00:00<00:39,  2.70it/s]

...

Boston_Celtics
The_Ten_Commandments_(1956_film)









  9%|▉         | 10/110 [00:01<00:20,  4.79it/s]

The_Ten_Commandments_(1956_film)
Tetris


  9%|▉         | 10/110 [00:01<00:19,  5.04it/s]


IndexError: ignored

In [0]:
tg = wiki.page('Nikolaj_Coster-Waldau').text.split(".")


In [0]:
tg[0:10]

['Nikolaj Coster-Waldau (Danish pronunciation: [ˈne̝koˌlɑjˀ ˈkʰʌstɐ ˈvæltɑw]; born 27 July 1970) is a Danish actor, producer and screenwriter',
 ' He graduated from the Danish National School of Performing Arts in Copenhagen in 1993',
 " Coster-Waldau's breakthrough performance in Denmark was his role in the film Nightwatch (1994)",
 ' Since then he has appeared in numerous films in his native Scandinavia and Europe in general, including Headhunters (2011) and A Thousand Times Good Night (2013)',
 '\nIn the U',
 'S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon',
 ' He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam (2008), as well as appearing as Frank Pike in the 2009 Fox television film Virtuality, originally intended as a pilot',
 ' He became widely known for his role as Jaime Lannister in the HBO fantasy series Game of Thrones, for which he received Primetime Emmy Award nomin

In [0]:
df_train['evidence'][10]

[[[269479, 265800, 'Tetris', 18]]]

In [0]:
text = pd.read_json(wiki_pages_folder+'/wiki-071.json', lines=True)

In [0]:
for i in range(len(text['id'])):
  if text['id'][i]== 'Nikolaj_Coster-Waldau':
    print(i)

25816


In [0]:
der = text['text'][25816].split(".")

In [0]:
der

['Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter ',
 ' He graduated from Danish National School of Theatre in Copenhagen in 1993 ',
 " Coster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- ",
 ' Since then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- ',
 '   In the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon ',
 ' He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot ',
 ' He became widely known to a broad audience for his current role as Ser Jaime Lannister , 

In [0]:
text.head()

,id,text,lines
0,"New_Caledonian_independence_referendum,_2018",An independence referendum will be held in New...,0\tAn independence referendum will be held in ...
1,Negaativa,Negaativa is a trio of female Brazilian Portug...,0\tNegaativa is a trio of female Brazilian Por...
2,"Nesøya,_Akershus",Nesøya is a small island in the municipality o...,0\tNesøya is a small island in the municipalit...
3,Newton_College_-LRB-Peru-RRB-,Newton College is a Peruvian British co-educat...,0\tNewton College is a Peruvian British co-edu...
4,National_Junior_Angus_Show,The National Junior Angus Show is the premier ...,0\tThe National Junior Angus Show is the premi...
